In [ ]:
import dis
from pepbenchmark.single_pred.base_dataset import SingleTaskDatasetManager
from pepbenchmark.utils.logging import disable_logging,enable_logging
# disable_logging()
enable_logging()
dataset = SingleTaskDatasetManager(
    dataset_name="test",
    official_feature_names=["fasta","label"],
    force_download=True,
)
dataset.set_official_feature("fasta")
dataset.set_official_feature("smiles")
print(f"Dataset length: {len(dataset)}")
print("Official features:", dataset.official_feature_dict.keys())
dataset.set_user_feature("other_feature", list(range(len(dataset))))
print("User features:", dataset.user_feature_dict.keys())

# Set split indices
dataset.set_official_split_indices(split_type="random_split", fold_seed=1)
print("Split indices:", dataset.get_split_indices())

# Get train, valid, test features
train_features, valid_features, test_features = dataset.get_train_val_test_features(format="pytorch_dataset")
print("Train features:", train_features[0])

: 